ERROR: Could not find a version that satisfies the requirement aimo (from versions: none)
ERROR: No matching distribution found for aimo


In [7]:
# custom client with custom model loader

class BertModelClient:
    def __init__(self, config, **kwargs):
        print(f"CustomModelClient config: {config}")
        self.device = config.get("device", "cpu")
        self.model = AutoModelForCausalLM.from_pretrained(config["model"]).to(self.device)
        self.model_name = config["model"]
        self.tokenizer = AutoTokenizer.from_pretrained(config["model"], use_fast=False)
        self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

        # params are set by the user and consumed by the user since they are providing a custom model
        # so anything can be done here
        gen_config_params = config.get("params", {})
        self.max_length = gen_config_params.get("max_length", 256)

        print(f"Loaded model {config['model']} to {self.device}")

    def create(self, params):
        if params.get("stream", False) and "messages" in params:
            raise NotImplementedError("Local models do not support streaming.")
        else:
            num_of_responses = params.get("n", 1)

            # can create my own data response class
            # here using SimpleNamespace for simplicity
            # as long as it adheres to the ClientResponseProtocol

            response = SimpleNamespace()

            inputs = self.tokenizer.apply_chat_template(
                params["messages"], return_tensors="pt", add_generation_prompt=True
            ).to(self.device)
            inputs_length = inputs.shape[-1]

            # add inputs_length to max_length
            max_length = self.max_length + inputs_length
            generation_config = GenerationConfig(
                max_length=max_length,
                eos_token_id=self.tokenizer.eos_token_id,
                pad_token_id=self.tokenizer.eos_token_id,
            )

            response.choices = []
            response.model = self.model_name

            for _ in range(num_of_responses):
                outputs = self.model.generate(inputs, generation_config=generation_config)
                # Decode only the newly generated text, excluding the prompt
                text = self.tokenizer.decode(outputs[0, inputs_length:])
                choice = SimpleNamespace()
                choice.message = SimpleNamespace()
                choice.message.content = text
                choice.message.function_call = None
                response.choices.append(choice)

            return response

    def message_retrieval(self, response):
        """Retrieve the messages from the response."""
        choices = response.choices
        return [choice.message.content for choice in choices]

    def cost(self, response) -> float:
        """Calculate the cost of the response."""
        response.cost = 0
        return 0

    @staticmethod
    def get_usage(response):
        # returns a dict of prompt_tokens, completion_tokens, total_tokens, cost, model
        # if usage needs to be tracked, else None
        return {}

In [1]:
class MathProblemSolver:
    def __init__(self, model_name, device):
        self.device = device
        self.model_name = model_name
        self.tokenizer, self.model = self.load_model_and_tokenizer()
        self.generation_config = self.load_generation_config()
        self.tokenizerBERT, self.modelBERT = self.load_bert_model()
        self.label_mapping = {
            0: "Algebra",
            1: "Counting & Probability",
            2: "Geometry",
            3: "Intermediate Algebra",
            4: "Number Theory",
            5: "Prealgebra",
            6: "Precalculus"
        }
          
    def load_model_and_tokenizer(self):
        try:
            tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if self.device == 'cuda' else torch.float32,
                device_map='auto',
                offload_folder="offload"
            )
            return tokenizer, model
        except Exception as e:
            print(f"Error loading model/tokenizer: {e}")
            raise

    def load_generation_config(self):
        try:
            generation_config = GenerationConfig.from_pretrained(self.model_name)
            generation_config.pad_token_id = self.tokenizer.eos_token_id
            return generation_config
        except Exception as e:
            print(f"Error loading generation config: {e}")
            raise
            
    def load_bert_model(self):
        modelname= '/kaggle/input/bert-classifier-math/transformers/prefinetuned/1/bert-finetuned-math-prob-classification'
        tokenizerBERT = AutoTokenizer.from_pretrained(modelname)
        modelBERT = AutoModelForSequenceClassification.from_pretrained(modelname)
        modelBERT.to(self.device)
        return tokenizerBERT, modelBERT

    def split_into_steps(self, text):
        try:
            sentences = text.split('. ')
            sentences = [s.strip() for s in sentences if s.strip()]
            steps = [f"Step {i+1}: {sentence}" for i, sentence in enumerate(sentences)]
            return steps
        except Exception as e:
            print(f"Error splitting instructions: {e}")
            return text

    def select_random_instructions(self, csv_file, class_label):
        try:
            df = pd.read_csv(csv_file)
            df = df[pd.to_numeric(df['answer'], errors='coerce').notnull()]
            df = df.drop_duplicates(subset=['problem'])
            filtered_df = df[df['Type'] == class_label.lower()]
            selected_rows = filtered_df.groupby('Type').head(2)
            results = [{'question': row['problem'], 'instruction': row['steps'], 'answer': row['answer']} for _, row in selected_rows.iterrows()]
            return results
        except Exception as e:
            print(f"Error selecting random instructions: {e}")
            return [{'question': '...', 'instruction': 'Step 1: ..., Step 2: ...', 'answer': '\\boxed{0-99}'}]

    def generate_answer_multiple_times(self, question, label_type, csv_file, num_iterations=N_REPETITIONS):
        exemplars = self.select_random_instructions(csv_file, label_type)
        exemplar_texts = "\n\n".join(
            f"Q: {exemplar['question']}\n"
            f"Steps: {self.split_into_steps(exemplar['instruction'])}\n"
            f"A: {exemplar['answer']}"
            for exemplar in exemplars
        )

        prompt = (
            'You are going to solve math problems that have a positive integer solution. Keep logic concise.'
            f"Here is a math problem you are to solve (positive numerical answer): {question}\n"
            f"This particular question is a {label_type} question.\n"
            "To solve it, first determine a series of logical steps for solving the problem and then follow these steps. It is imperative that the final answer is in the brackets of \\boxed{}. \n\n"
            "The final answer should be a positive integer and not an algebraic expression.\n"
            f"Here are some examples of how to solve similar {label_type} problems step-by-step:\n\n"
            f"{exemplar_texts}\n\n"
            "Answer:"
        )

        answers = []
        for _ in range(num_iterations):
            try:
                inputs = self.tokenizer(prompt, return_tensors='pt').to(self.device)
                with torch.no_grad():
                    outputs = self.model.generate(
                        inputs['input_ids'],
                        max_new_tokens=MAX_NEW_TOKENS,
                        num_beams=3,
                        early_stopping=True,
                        no_repeat_ngram_size=2,
                        attention_mask=inputs['attention_mask'],
                        temperature=0.7,
                        top_p=0.9,
                        top_k=50
                    )

                result = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
#                 match = re.search(r'Answer:\s*(.*)', result, re.DOTALL)
                match = self.process_text_output(result)
                if match:
                    answers.append(match)
#                     answers.append(self.extract_boxed_answer(match.group(1).strip()))
            except Exception as e:
                print(f"Error during generation: {e}")

        return answers

#     def extract_boxed_answer(self, generated_text):
#         try:
#             match = re.search(r'\\boxed{([^}]*)}', generated_text)
#             if match:
#                 return match.group(1)
#         except Exception as e:
#             print(f"Error extracting boxed answer: {e}")
#         return 0  
    
    def naive_parse(self, answer):
        try:
            out = []
            for l in list(answer):
                if l in '0123456789':
                    out.append(l)
            return ''.join(out)
        except Exception as e:
            return -1

    def process_text_output(self, output):
        try:
            result_output = re.findall(r'\\boxed\{(\d+)\}', output)
            if not result_output:  #no boxed answer given
                result_output = self.naive_parse(output)
            else:
                result_output = result_output[-1] #last instance of boxed answer

#             if not result_output:
#                 result_output = -1
            
            result_output = round(float(eval(result_output))) % 1000
        except Exception as e:
            print(e)
            result_output = -1
        return result_output    # int

    def aggregate_answers(self, answers):
        answer_counts = {}
        for answer in answers:
            if answer in answer_counts:
                answer_counts[answer] += 1
            else:
                answer_counts[answer] = 1
        sorted_answers = sorted(answer_counts.items(), key=lambda item: item[1], reverse=True)
        return sorted_answers[0] if sorted_answers else (None, 0)

    def manage_context_and_generate_answers(self, question, context_prompt, total_tokens, conversation, label_type, csv_file):
        try:
            answers = self.generate_answer_multiple_times(question, label_type, csv_file) #list of integers
            best_answer, count = self.aggregate_answers(answers) 
#             best_generated_text = best_answer
#             boxed_answer = self.process_text_output(best_answer)
            qna_text = f"Question: {question}\nAnswer: {best_answer}\n\n"
            conversation += qna_text
            total_tokens += len(self.tokenizer.encode(qna_text))

            if total_tokens >= 4000:
                conversation += context_prompt + "\n\n"
                total_tokens = len(self.tokenizer.encode(conversation))

#             result = {
# #                 'generated_text': answers,
#                 'int_answer': best_answer,
# #                 'total_tokens': total_tokens
#             }
            result = best_answer
            print(result)
            return result
        except Exception as e:
            return {'generated_text': '', 'int_answer': 1, 'total_tokens': 4001}

    def predict(self, text):
        try:
            inputs = self.tokenizerBERT(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
            inputs = {key: val.to(self.device) for key, val in inputs.items()}

            with torch.no_grad():
                outputs = self.modelBERT(**inputs)

            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_class = torch.argmax(predictions, dim=-1).item()
            return self.label_mapping[predicted_class]
        except Exception as e:
            predicted_class = random.randint(0,6)
            return self.label_mapping[predicted_class]

    def flush(self):
        torch.cuda.empty_cache()
        gc.collect()

NameError: name 'N_REPETITIONS' is not defined

In [9]:
context_prompt = """
This is a compendium of things which we would normally explain in maths competitions for teenaged schoolchildren as context for you to better understand the task at hand:

\\begin{enumerate}
    \\item The final answer should be a positive integer and should be placed within $\\boxed{}$.
    \\item Established mathematical notation will be used.
    \\item We might use a colon or a vertical line as a separator in set notation so $\\{x \\mid x \\in \\mathbb{Z}, x > 0\\} = \\{y : y \\in \\mathbb{Z}, y > 0\\}$.
    \\item Floor and ceiling notation, so for $x$ a real number we let $\\lfloor x \\rfloor = \\max\\{z \\mid x \\in \\mathbb{Z}, z \\leq x\\}$. Similarly $\\lceil x \\rceil = \\min\\{z \\mid x \\in \\mathbb{Z}, z \\geq x\\}$.
    \\item Fractional part notation. If $x$ is a real number, we define $\\{x\\}$ to mean $x - \\lfloor x \\rfloor$.
    \\item We write a line over a non-negative integer written in base 10 notation to indicate that it is being viewed as a string of digits rather than a number. Thus the second digit of $\\overline{1729}$ is 7 but 1729 does not have a second digit because it is an integer.
    \\item We allow a phrase such as "$x$ is a 3-digit positive integer" to mean that if written in Arabic notation as $x = a_m \\cdot a_{m-1} \\cdot \\ldots \\cdot a_1$ with $a_i$ all digits and $a_m \\neq 0$, then $n = m$. We allow "the sum of the digits of $n$" to mean: write $n$ in Arabic base 10 notation and then sum the digits.
    \\item We allow informal probability language such as: "a point is chosen uniformly at random in the interval $[0, 1]$".
    \\item We use $\\binom{n}{r}$ to denote the number of ways of choosing $r$ things from $n$ things.
    \\item The sum over the empty set is 0 and the product over the empty set is 1.
    \\item We use an ellipsis to denote an obvious pattern, either on the line of print of midline (as appropriate) so the set of the first $n$ positive integers can be written $\\{1, 2, \\ldots, n\\}$ and their sum is $1 + 2 + \\ldots + n$.
    \\item For integers $l$, $m$, $n$ then $l$ raised by $m$ which is raised by $n$ denotes $l^{(m^n)}$.
    \\item $m^0 = 1$ for all integers $m$ (including 0) if doing combinatorial enumeration. If $x$ is real then $x^0$ needs to be clarified if $x = 0$.
    \\item British or American versions of English can be used. Thus "highest common factor" means the same as "greatest common divisor".
    \\item A prefix or subscript may be used to indicate features of a triangle associated with vertices. Thus triangle $ABC$ has three altitudes, and the one dropped from $A$ could be denoted the altitude through $A$, the $A$-altitude or the altitude $h_a$. Similarly for median lines.
    \\item If the term natural number is used, then it will be made clear if 0 is a natural number.
    \\item $:=$ means 'is defined to be equal to'.
\\end{enumerate>"""

total_tokens = 0
full_conversation = context_prompt + "\n\n"

# Set up the evaluation API
import aimo

env = aimo.make_env()
iter_test = env.iter_test()

inputfile= '/kaggle/input/prmclean/prmclean.csv'
solver = MathProblemSolver(MODEL_NAME, DEVICE)

ModuleNotFoundError: No module named 'aimo'